In [1]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs

# Data

## Case1

In [2]:
@show dsl = Dataset(xid = [111,222,333,444,555], 
                    x1 = [1,2,1,4,5], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl = Dataset(xid = [111, 222, 333, 444, 555], x1 = [1, 2, 1, 4, 5], x2 = [-1.2, -3, 2.1, -3.5, 2.8], x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")], x4 = ["abcd", "efgh", "ijkl", "mnop", "qrst"]) = 5×5 Dataset
 Row │ xid       x1        x2        x3          x4
     │ identity  identity  identity  identity    identity
     │ Int64?    Int64?    Float64?  Date?       String?
─────┼────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd
   2 │      222         2      -3.0  2019-09-30  efgh
   3 │      333         1       2.1  2019-10-04  ijkl
   4 │      444         4      -3.5  2019-10-03  mnop
   5 │      555         5       2.8  2019-10-03  qrst
dsr = Dataset(yid = [111, 111, 222, 444, 333], y1 = [3, 3, 3, 3, 3], y2 = [0, 0, -3, 1, 2], y3 = [Date("2019-10-01"), Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")], y4 = ["abc", "abcd", "efg", "mnop",

## Case2

In [18]:
dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

## Case3

In [19]:
dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

# Version-3

In [ ]:
function my_cartesianjoin_v3(dsl::AbstractDataset, dsr::AbstractDataset;
  on=nothing, threads::Bool=true,
  flag=ones(Bool, nrow(dsl), nrow(dsr)))

  reset_timer!()

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on)
    if typeof(element) <: Pair
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    else
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    end
  end

  if Set(conditions) == Set([isequal])
    return IMD.innerjoin(dsl, dsr, on=on)
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)

  equalon_dsr_cols = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)
  right_cols = setdiff(1:length(IMD.index(dsr)), equalon_dsr_cols)

  oncols_left = onleft
  oncols_right = onright


  # get flag, idx and ranges
  #println(flag)
  @timeit "compute flag matrix"  cross_compare_2(dsl, dsr, flag, conditions, onleft, onright, threads)
  #cross_compare(dsl,dsr,flag,conditions,onleft,onright,threads)
  # println(flag)
  ## return flag
  @timeit "compute idx and ranges" idx, ranges = handle_flag_2(dsl, dsr, sparse(flag), threads)
  # idx, ranges = handle_flag_t(dsl,dsr,sparse(flag),threads)
  """
  println("idx,ranges")
  println(idx)
  println(ranges)
  """

  # create result
  @timeit "generate_newds" newds = generate_newds(idx, ranges, dsl, dsr, right_cols, threads)

  print_timer()
  newds ,idx,ranges
end

function cross_compare(dsl, dsr,
  flag, conditions, onleft, onright, threads)
  oncols_left = onleft
  oncols_right = onright

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  for i in 1:length(conditions)  # Each conditions 每个条件
    fun = conditions[i]
    #println(var_r)
    # tmp = zeros(r_len)
    @_threadsfor threads for j in 1:l_len # multithread safe? # 可以多线程 逐行判断
      #flag[j,:] =  flag[j,:] .* fun.((IMD._columns(dsl)[oncols_left[i]][j]), IMD._columns(dsr)[oncols_right[i]])
      flag[j, :] = flag[j, :] .* _op_for_vector((IMD._columns(dsl)[oncols_left[i]][j]), IMD._columns(dsr)[oncols_right[i]], fun)
    end
  end

  flag
end

Base.@propagate_inbounds function _op_for_vector(value, vector, fun)
  fun.(value, vector)
end

Base.@propagate_inbounds function _op_for_vector_2(value, vector, fun)
  res = Array{Bool}(undef,length(vector))
  for i in 1:length(vector)
    res[i] = fun(value,vector[i])
  end
  res
end

function cross_compare_2(dsl, dsr,
  flag, conditions, onleft, onright, threads)
  oncols_left = onleft
  oncols_right = onright

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  for i in 1:length(conditions)  # Each conditions 每个条件
    fun = conditions[i]
    @_threadsfor threads for j in 1:l_len # multithread safe? # 可以多线程 逐行判断
      #println(_op_for_vector((IMD._columns(dsl)[oncols_left[i]][j]), IMD._columns(dsr)[oncols_right[1]], fun))
      update_row!(view(flag,j,:), _op_for_vector_2((IMD._columns(dsl)[oncols_left[i]][j]), IMD._columns(dsr)[oncols_right[i]], fun))
    end
  end

  #println(flag)
  flag
end

function update_row!(row,res)
  row .*= res
end

function handle_flag(dsl, dsr, flag)
  #flag = sparse(flag)
  idx = Int32[]
  ranges = Vector{UnitRange{Int32}}(undef, nrow(dsl))
  lo_idx = 1

  
  for j in 1:nrow(dsl)  # 应该也可以多线程
    dsr_satis_idx = findall(isone, flag[j, :])   # 每一行的对应索引

    if length(dsr_satis_idx) != 0
      if !(dsr_satis_idx[1] in idx)   # first?
        idx = [idx; dsr_satis_idx]    # 放到idx里，现在认为不会有跨值的情况 .first和.end不同时存在或不存在
      end
      lo_idx = findfirst(isequal(dsr_satis_idx[1]), idx)  # 算range
      hi_idx = lo_idx + length(dsr_satis_idx) - 1
      ranges[j] = lo_idx:hi_idx
    else
      ranges[j] = lo_idx:(lo_idx-1)
    end
  end

  
  append!(idx, setdiff(1:nrow(dsr), idx))
  idx, ranges
end

# invalid
function handle_flag_t(dsl, dsr, flag, threads)
  #flag = sparse(flag)
  idx = Int32[]
  ranges = Vector{UnitRange{Int32}}(undef, nrow(dsl))
  lo_idx = 1

  @_threadsfor threads for i in 1:nrow(dsl)  # 应该也可以多线程？
    dsr_satis_idx = findall(isone, flag[i, :])   # 每一行的对应索引

    if length(dsr_satis_idx) != 0
      if !(dsr_satis_idx[1] in idx)   # first?
        idx = [idx; dsr_satis_idx]  # 放到idx里，现在认为不会有跨值的情况 .first和.end不同时存在或不存在
      end
      lo_idx = findfirst(isequal(dsr_satis_idx[1]), idx)  # 算range
      hi_idx = lo_idx + length(dsr_satis_idx) - 1
      ranges[i] = lo_idx:hi_idx
    else
      ranges[i] = lo_idx:(lo_idx-1)
    end
  end

  append!(idx, setdiff(1:nrow(dsr), idx))
  idx, ranges
end

function handle_flag_2(dsl, dsr, flag,threads::Bool = false)
  # 没有聚合ranges 但是空间换时间

  onesres = findall(isone, transpose(flag))
  idx = getindex.(onesres,1)

  ranges = Vector{UnitRange{Int64}}(undef, nrow(dsl))
  fill!(ranges,1:0)
  
  A = getindex.(onesres,2)
  B = hcat([[i, count(==(i), A)] for i in unique(A)]...)[2,:]  # count each idx
  C = cumsum(B)
  D = [1;C .+ 1]
   """
  println(onesres)
  println("ABCD")
  println(A)
  println(B)
  println(C)
  println(D)
 """
  loc = 1
  @_threadsfor false for i in unique(A)  # i的位置被随机了，和loc对不上，不能多线程
    # rintln(loc)
    ranges[i] = D[loc]:C[loc]
    loc += 1
    ##println(ranges)
  end


 #println(idx)
 #println(ranges)

 idx, ranges
end

function generate_newds(idx, ranges, dsl, dsr, right_cols, threads::Bool=false)
  new_ends = map(length, ranges)
  IMD.our_cumsum!(new_ends)
  total_length = new_ends[end]

  inbits = nothing
  revised_ends = nothing
  #threads = true
  makeunique = false

  res = []
  for j in 1:length(IMD.index(dsl))
    _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)
    if IMD.DataAPI.refpool(_res) !== nothing
      IMD._fill_oncols_left_table_inner!(_res.refs, IMD.DataAPI.refarray(IMD._columns(dsl)[j]), ranges, new_ends, total_length; inbits=inbits, en2=revised_ends, threads=threads)
    else
      IMD._fill_oncols_left_table_inner!(_res, IMD._columns(dsl)[j], ranges, new_ends, total_length; inbits=inbits, en2=revised_ends, threads=false)
    end
    push!(res, _res)
  end

  if dsl isa SubDataset
    newds = Dataset(res, copy(IMD.index(dsl)), copycols=false)
  else
    newds = Dataset(res, IMD.Index(copy(IMD.index(dsl).lookup), copy(IMD.index(dsl).names), copy(IMD.index(dsl).format)), copycols=false)
  end

  for j in 1:length(right_cols)
    _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)
    if IMD.DataAPI.refpool(_res) !== nothing
      IMD._fill_right_cols_table_inner!(_res.refs, view(IMD.DataAPI.refarray(IMD._columns(dsr)[right_cols[j]]), idx), ranges, new_ends, total_length; inbits=inbits, en2=revised_ends, threads=threads)
    else
      IMD._fill_right_cols_table_inner!(_res, view(IMD._columns(dsr)[right_cols[j]], idx), ranges, new_ends, total_length; inbits=inbits, en2=revised_ends, threads=threads)
    end
    push!(IMD._columns(newds), _res)

    new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=makeunique)[end]
    push!(IMD.index(newds), new_var_name)
    setformat!(newds, IMD.index(newds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))
  end

  
  newds
end


# Cases Test

# Version-4

In [82]:
function my_cartesianjoin_v4(dsl::AbstractDataset, dsr::AbstractDataset;
  on=nothing, threads::Bool=true, flag = ones(Bool,nrow(dsl) * nrow(dsr)))

  reset_timer!()

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on)
    if typeof(element) <: Pair
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    else
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    end
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)

  equalon_dsr_cols = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)
  right_cols = setdiff(1:length(IMD.index(dsr)), equalon_dsr_cols)

  oncols_left = onleft
  oncols_right = onright

  #println(oncols_left)
  #println(right_cols)

  # get flag, idx and ranges
  #println(flag)
  @timeit "compute flag vector"  cross_compare_vec(dsl, dsr, flag, conditions, onleft, onright, threads)
  #cross_compare(dsl,dsr,flag,conditions,onleft,onright,threads)
  #println(flag)

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  @timeit "new ds" nds = generate_newds_v4(flag,dsr,dsr,l_len,r_len,right_cols)

  println()
  print_timer()
  flag,nds
end

function cross_compare_vec(dsl, dsr,
  flag, conditions, onleft, onright, threads)

  oncols_left = onleft
  oncols_right = onright

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  for i in 1:length(conditions)  # Each conditions 每个条件
    fun = conditions[i]

    IMD.@_threadsfor threads for j in 1:l_len  # each row in dsl
      cur_index = (j-1)*l_len  
      ## 传递函数的allocations？
      _op_for_dsrcol(flag,fun,cur_index,IMD._columns(dsl)[oncols_left[i]][j],IMD._columns(dsr)[oncols_right[i]],r_len,
                     oncols_left,oncols_right)
      
    end
  end
  
end

function _op_for_dsrcol(flag,fun,cur_index,x,r_col,r_len,oncols_left,oncols_right)
  for k in 1:r_len
    flag[cur_index+k] == 0 && continue
    #println(IMD._columns(dsl)[oncols_left[i]][j],",",cur_index,",",IMD._columns(dsr)[oncols_right[i]][k])
    flag[cur_index+k] &= fun(x,r_col[k])         
  end
end


function generate_newds_v4(flag,dsl,dsr,l_len,r_len,right_cols)

  @time begin
      ## new left
      dsl_count = []  # 每一行对应几个右边
      for i in 1:l_len
          lo = 1+(i-1)*r_len
          hi = lo + r_len - 1
          push!(dsl_count,count(==(true),flag[lo:hi]))
      end
      
      new_ends = IMD.our_cumsum!(dsl_count)  # 累计和
      total_length = new_ends[end]
      
      res = []
      for j in 1:length(IMD.index(dsl))  # left 的每一列
        _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)  # 
        
        x = IMD._columns(dsl)[j] # 这一步可以隔离
      
        # 左侧填充
        IMD.@_threadsfor true for i in 1:length(x)
          dsl_count[i] == 0 && continue
          i == 1 ? lo = 1 : lo = new_ends[i - 1] + 1
          hi = new_ends[i]
          IMD._fill_val_join!(_res, lo:hi, x[i])
        end
        
        push!(res, _res)
      end
  
      if dsl isa SubDataset
        newds = Dataset(res, copy(IMD.index(dsl)), copycols=false)
      else
        newds = Dataset(res, IMD.Index(copy(IMD.index(dsl).lookup), copy(IMD.index(dsl).names), copy(IMD.index(dsl).format)), copycols=false)
      end
  end

  
  @time begin
  ## new right
  #println("Cerating right")

  dsr_idx = []
  for i in 1:l_len
      lo = 1+(i-1)*r_len
      hi = lo + r_len - 1
      append!(dsr_idx,findall(isone,flag[lo:hi]))
  end 

  #println(dsr_idx)

  #println("Nesting")
  for j in 1:length(right_cols)
    _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)  # 空的dsr

    IMD.@_threadsfor true for i in 1:length(l_len)
      #print(j)
      length(dsr_idx[i]) == 0 && continue
      for i in 1:length(dsr_idx)
        _res[i] = IMD._columns(dsr)[right_cols[j]][dsr_idx[i]]
      end
    end

    #println(_res)
    push!(IMD._columns(newds), _res)

    new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=true)[end]
    push!(IMD.index(newds), new_var_name)
    setformat!(newds, IMD.index(newds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))
  end
end
  newds

  
end

generate_newds_v4 (generic function with 1 method)

In [6]:
function fun1(x,y)  # UDF 每次都要编译
  x <= y
end

fun1 (generic function with 1 method)

In [84]:
flag,nds = my_cartesianjoin_v4(dsl,dsr,
  on = [:xid=>:yid, :x2=>:y2=>fun1]);

  0.000643 seconds (297 allocations: 22.859 KiB)
  0.000542 seconds (376 allocations: 29.375 KiB)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           1.62ms /  96.5%           68.8KiB /  94.7%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1   1.37ms   87.9%  1.37ms   56.7KiB   87.0%  56.7KiB
 compute flag vector        1    189μs   12.1%   189μs   8.47KiB   13.0%  8.47KiB
 ────────────────────────────────────────────────────────────────────────────────


In [64]:
nds

4×9 Dataset
 Row │ yid       y1        y2        y3          y4        y1_1      y2_1      ⋯
     │ identity  identity  identity  identity    identity  identity  identity  ⋯
     │ Int64?    Int64?    Int64?    Date?       String?   Int64?    Int64?    ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         3         0  2019-10-01  abc              3         0  ⋯
   2 │      111         3         0  2019-10-01  abc              3         0
   3 │      111         3         0  2019-10-01  abcd             3        -3
   4 │      444         3         1  2019-10-05  mnop             3         1
                                                               2 columns omitted

In [66]:
@time my_cartesianjoin_v4(dsl_big,dsr_big,
  on = [:x1 => :y1, :x2 => :y2 => fun1],threads=true);

  0.001405 seconds (13.73 k allocations: 1.573 MiB)
  0.003285 seconds (39.46 k allocations: 6.317 MiB)
 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           6.64ms /  99.4%           7.98MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1   4.88ms   74.0%  4.88ms   7.89MiB   98.9%  7.89MiB
 compute flag vector        1   1.71ms   26.0%  1.71ms   86.3KiB    1.1%  86.3KiB
 ────────────────────────────────────────────────────────────────────────────────
  0.008719 seconds (59.19 k allocations: 8.970 MiB)


In [89]:
@time newds = my_cartesianjoin_v4(dsl_big2,dsr_big2,
  on = [:x1 => :y1, :x2 => :y2 => fun1],threads=true);

  0.044469 seconds (140.22 k allocations: 121.659 MiB)


  0.225404 seconds (3.55 M allocations: 231.872 MiB, 18.10% gc time)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            368ms / 100.0%            354MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1    270ms   73.5%   270ms    354MiB   99.7%   354MiB
 compute flag vector        1   97.4ms   26.5%  97.4ms    930KiB    0.3%   930KiB
 ────────────────────────────────────────────────────────────────────────────────
  0.403273 seconds (3.75 M allocations: 449.851 MiB, 14.76% gc time)


## left newds

In [138]:
idx = [1,2,3,4,5]
ranges = [1:2,3:3,1:0,4:4,1:0]
res = []

new_ends = map(length, ranges)
IMD.our_cumsum!(new_ends)
total_length = new_ends[end]

for j in 1:length(IMD.index(dsl))  # left 的每一列
  _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)
  if IMD.DataAPI.refpool(_res) !== nothing
    IMD._fill_oncols_left_table_inner!(_res.refs, IMD.DataAPI.refarray(IMD._columns(dsl)[j]), ranges, new_ends, total_length; inbits=nothing, en2=nothing, threads=threads)
  else
    IMD._fill_oncols_left_table_inner!(_res, IMD._columns(dsl)[j], ranges, new_ends, total_length; inbits=nothing, en2=nothing, threads=false)
  end
  push!(res, _res)
end


In [169]:
res

5-element Vector{Any}:
 Union{Missing, Int64}[111, 111, 222, 444]
 Union{Missing, Int64}[1, 1, 2, 4]
 Union{Missing, Float64}[-1.2, -1.2, -3.0, -3.5]
 Union{Missing, Date}[Date("2019-10-03"), Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-03")]
 Union{Missing, String}["abcd", "abcd", "efgh", "mnop"]

In [196]:
new_ends

5-element Vector{Int64}:
 2
 3
 3
 4
 4

In [187]:
en = new_ends
x = IMD._columns(dsl)[1]
IMD.@_threadsfor true for i in 1:length(x)
  length(ranges[i]) == 0 && continue
  i == 1 ? lo = 1 : lo = en[i - 1] + 1
  hi = en[i]
  IMD._fill_val_join!(_res, lo:hi, x[i])
end

In [214]:
l_len = nrow(dsl)
r_len = nrow(dsr)

5

In [219]:
new_ends

5-element Vector{Any}:
 2
 1
 0
 1
 0

In [206]:
lo = 1+(3-1)*r_len
hi = lo + r_len
count(==(true),flag[lo:hi])

0

In [216]:
new_ends = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    push!(new_ends,count(==(true),flag[lo:hi]))
end

5-element Vector{Int64}:
 2
 1
 0
 1
 0

In [231]:

dsl_count = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    push!(dsl_count,count(==(true),flag[lo:hi]))
end

new_ends = IMD.our_cumsum!(dsl_count)
total_length = new_ends[end]

res = []
for j in 1:length(IMD.index(dsl))  # left 的每一列
  _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)  # 
  
  x = IMD._columns(dsl)[j] # 这一步可以隔离

  IMD.@_threadsfor true for i in 1:length(x)
    dsl_count[i] == 0 && continue
    i == 1 ? lo = 1 : lo = en[i - 1] + 1
    hi = en[i]
    IMD._fill_val_join!(_res, lo:hi, x[i])
  end
  
  push!(res, _res)
end

In [321]:
res

5-element Vector{Any}:
 Union{Missing, Int64}[111, 111, 222, 444]
 Union{Missing, Int64}[1, 1, 2, 4]
 Union{Missing, Float64}[-1.2, -1.2, -3.0, -3.5]
 Union{Missing, Date}[Date("2019-10-03"), Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-03")]
 Union{Missing, String}["abcd", "abcd", "efgh", "mnop"]

In [361]:
IMD._columns(dsl)[1] 

5-element Vector{Union{Missing, Int64}}:
 111
 222
 333
 444
 555

## right newds

In [405]:
right_cols = [2,3,4,5]

4-element Vector{Int64}:
 2
 3
 4
 5

In [411]:
dsl_count = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    push!(dsl_count,count(==(true),flag[lo:hi]))
end

new_ends = IMD.our_cumsum!(dsl_count)
total_length = new_ends[end]

for j in 1:length(right_cols)
  _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)  # 空的dsr
  
  IMD._fill_right_cols_table_inner!(_res, view(IMD._columns(dsr)[right_cols[j]], idx), 
                  ranges, new_ends, total_length; inbits=nothing, en2=nothing, threads=true)

  # push!(IMD._columns(newds), _res)
  #new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=makeunique)[end]
  #  push!(IMD.index(newds), new_var_name)
  #  setformat!(newds, IMD.index(newds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))
  println(_res)
end

Union{Missing, Int64}[3, 3, 3, 3]
Union{Missing, Int64}[0, 0, -3, 1]
Union{Missing, Date}[Date("2019-10-01"), Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05")]
Union{Missing, String}["abc", "abcd", "efg", "mnop"]


In [445]:
_res = IMD.allocatecol(IMD._columns(dsr)[right_cols[1]], total_length, addmissing=false)  # 空的dsr


4-element Vector{Union{Missing, Int64}}:
 missing
 missing
 missing
 missing

In [444]:
_res

4-element Vector{Union{Missing, Int64}}:
 missing
 missing
 missing
 missing

In [ ]:
view(IMD._columns(dsr)[right_cols[2]],[1,2,3,4,5])
idx = [1,2,3,4,5]
ranges = [1:2,3:3,1:0,4:4,1:0]

In [350]:
findall(isone,flag[1:5])

2-element Vector{Int64}:
 1
 2

In [359]:
IMD._columns(dsr)[right_cols[2]]

2-element Vector{Union{Missing, Int64}}:
 0
 0

In [387]:
_res = IMD.allocatecol(IMD._columns(dsr)[right_cols[1]], total_length, addmissing=false)  # 空的dsr

@time begin
  for i in [1,2,3]
    copyto!(_res, 1, IMD._columns(dsr)[right_cols[2]], i, i)
  end
end

@time begin
  for i in [1,2,3]
    _res[i] = IMD._columns(dsr)[right_cols[2]][i]
  end
end

  0.000008 seconds (1 allocation: 80 bytes)
  0.000012 seconds (1 allocation: 80 bytes)


In [393]:
IMD._columns(dsr)[right_cols]

4-element Vector{AbstractVector}:
 Union{Missing, Int64}[3, 3, 3, 3, 3]
 Union{Missing, Int64}[0, 0, -3, 1, 2]
 Union{Missing, Date}[Date("2019-10-01"), Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")]
 Union{Missing, String}["abc", "abcd", "efg", "mnop", "qrst"]

In [417]:
dsr_idx = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    append!(dsr_idx,findall(isone,flag[lo:hi]))
end 

In [418]:
dsr_idx

4-element Vector{Any}:
 1
 2
 3
 4

In [424]:
_res = IMD.allocatecol(IMD._columns(dsr)[right_cols[1]], total_length, addmissing=false)  # 空的dsr
IMD.@_threadsfor true for i in 1:length(l_len)
  length(dsr_idx[i]) == 0 && continue
  for i in 1:length(dsr_idx)
    _res[i] = IMD._columns(dsr)[right_cols[1]][dsr_idx[i]]
  end
end

In [457]:
dsl_count = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    push!(dsl_count,count(==(true),flag[lo:hi]))
end

new_ends = IMD.our_cumsum!(dsl_count)
total_length = new_ends[end]

dsr_idx = []
for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    append!(dsr_idx,findall(isone,flag[lo:hi]))
end 

for j in 1:length(right_cols)
  _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)  # 空的dsr

  IMD.@_threadsfor true for i in 1:length(l_len)
    length(dsr_idx[i]) == 0 && continue
    for i in 1:length(dsr_idx)
      _res[i] = IMD._columns(dsr)[right_cols[j]][dsr_idx[i]]
    end
  end

  println(_res)

  push!(IMD._columns(nds), _res)

  new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=false)[end]
  push!(IMD.index(nds), new_var_name)
  setformat!(nds, IMD.index(nds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))

end

Union{Missing, Int64}[3, 3, 3, 3]
Union{Missing, Int64}[0, 0, -3, 1]
Union{Missing, Date}[Date("2019-10-01"), Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05")]
Union{Missing, String}["abc", "abcd", "efg", "mnop"]


In [453]:
push!(IMD.index(nds), new_var_name)

InMemoryDatasets.Index(Dict(:x1 => 2, :y1 => 6, :x2 => 3, :x3 => 4, :x4 => 5, :xid => 1), [:xid, :x1, :x2, :x3, :x4, :y1], Dict{Int64, Function}(), Int64[], Bool[], Base.RefValue{Bool}(false), Int64[], Int64[], Base.RefValue{Int64}(1), Base.RefValue{Bool}(false))

In [458]:
nds

4×9 Dataset
 Row │ xid       x1        x2        x3          x4        y1        y2        ⋯
     │ identity  identity  identity  identity    identity  identity  identity  ⋯
     │ Int64?    Int64?    Float64?  Date?       String?   Int64?    Int64?    ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd             3         0  ⋯
   2 │      111         1      -1.2  2019-10-03  abcd             3         0
   3 │      222         2      -3.0  2019-09-30  efgh             3        -3
   4 │      444         4      -3.5  2019-10-03  mnop             3         1
                                                               2 columns omitted